In [8]:
import numpy as np
from scipy.stats import norm

In [9]:
def dydx(func, x, d = 1e-5):
    return (func(x+d) - func(x))/d

In [10]:
def newton_solve(
        func,
        guess = 0.0,
        tol = 1e-5,
        max_iter = 10000
):
    """
    Jump in the opposite direction of the derivative until very close to root.
    1) Start by evaluating tangent line func(guess) to get the line's root
    2) 0 = func(guess) + func'(guess)(new_guess - guess)   <=== solve 0 = first order Taylor Expansion
        *  new_guess =  guess - ( func(guess) / func'(guess) ) 
    3) repeat until y < tol
    """
    temp_guess = guess

    for i in range(0,max_iter):
        if abs(func(temp_guess)) < tol:
            return temp_guess
        change = dydx(func, temp_guess)
        new_guess = temp_guess - func(temp_guess) / change
        temp_guess = new_guess

    return temp_guess


In [11]:
def black_scholes_call(S_0, K, sigma, T, r=0.05):
    """
    C = e^-rt [F * N(d1) - K * N(d2)] 
    """
    disc = np.exp(-r * T)
    F = S_0 * np.exp(r * T)
    d1 = np.log(F/K) / (sigma * np.sqrt(T)) + (sigma * np.sqrt(T))/2
    d2 = d1 - sigma * np.sqrt(T)

    return disc * (F * norm.cdf(d1) - K * norm.cdf(d2))

def black_scholes_put(S_0, K, sigma, T, r=0.05):
    """
    P = e^-rt [K * N(-d2) - F * N(-d1)] 
    """
    disc = np.exp(-r * T)
    F = S_0 * np.exp(r * T)
    d1 = np.log(F/K) / (sigma * np.sqrt(T)) + (sigma * np.sqrt(T))/2
    d2 = d1 - sigma * np.sqrt(T)

    return disc * (K * norm.cdf(- d2) - F * norm.cdf(-d1))

In [12]:
def implied_vol(market_price, S_0, K, T, r=0.05):
    """
    In theory market_price - BS_price = 0
    """
    func = lambda sigma : black_scholes_call(S_0=S_0, K=K, T=T, sigma = sigma, r=r) - market_price

    return newton_solve(func, guess=0.15)

In [13]:
def put_call_parity(S_0, K, price, r, T, call_to_put = True):
    """
    C = P + F e-rT 
      = P + S_O - k e^-rT

    P = C - S_0 + k e^-rT
    """
    if call_to_put:
        C = price
        return C - S_0 + K * np.exp(-r * T)
    else:
        P = price
        return P + S_0 - K * np.exp(-r * T)

In [ ]:
print("Black-Scholes Call Price:")
print(black_scholes_call(40,40,.22, 1, 0.04))

market_price, S, K, T, r = 100, 215, 225, 5,0.05
iv = implied_vol(market_price, S, K, T, r)
print("Implied Volatility:")
print(iv)

print("Put-Call Parity Check:")
call_price = black_scholes_call(S, K, iv, T, r)
put_actual = black_scholes_put(S, K, iv, T, r)
put_price = put_call_parity(S, K, call_price, r, T, call_to_put=True)
print(f"Put Price: {put_price}, Actual Put Price: {put_actual}")

Black-Scholes Call Price:
4.275529177903377
Implied Volatility:
0.47394845847033795
Put-Call Parity Check:
Put Price: 60.23017510891799, Actual Put Price: 60.230175108917976
